# Recommendation Engine
Online Challenge: Build A Recommendation Engine 

## 1. Set Credentials for IBM cloud

In [49]:
# The code was removed by Watson Studio for sharing.

## 2. Load Data

### 2.1 Train data

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_train = spark.read\
    .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
    .option('header', 'true')\
    .load(cos.url('train.csv', 'recommendationengine-donotdelete-pr-vknxjbiqabtc3s'))

df_train.createOrReplaceTempView('df_train')
df_train.show()

+----------+---------+--------+------------+---------+-------+---------+
|CustomerID|InvoiceNo|Quantity| InvoiceDate|UnitPrice|Country|StockCode|
+----------+---------+--------+------------+---------+-------+---------+
|     27270|    27270|       7|1/12/10 8:26|     2.55|     PX|  85123AY|
|     27270|    27270|       7|1/12/10 8:26|     3.39|     PX|   71053R|
|     27270|    27270|       9|1/12/10 8:26|     2.75|     PX|  84406BH|
|     27270|    27270|       7|1/12/10 8:26|     3.39|     PX|  84029GV|
|     27270|    27270|       7|1/12/10 8:26|     3.39|     PX|  84029EX|
|     27270|    27270|       2|1/12/10 8:26|     7.65|     PX|   22752G|
|     27270|    27270|       7|1/12/10 8:26|     4.25|     PX|   21730R|
|    427266|   427266|       3|1/12/10 8:35|     5.95|     PX|   21756Q|
|    462735|   462735|      28|1/12/10 8:45|     3.75|     RA|   22728B|
|    462735|   462735|      28|1/12/10 8:45|     3.75|     RA|   22727B|
|    462735|   462735|      14|1/12/10 8:45|     3.

### 2.2 Test data

In [4]:
df_test = spark.read\
    .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
    .option('header', 'true')\
    .load(cos.url('test.csv', 'recommendationengine-donotdelete-pr-vknxjbiqabtc3s'))

df_test.createOrReplaceTempView('df_test')
df_test.show()

+-------+----------+------------+---------+--------+---------+---------+
|Country|CustomerID| InvoiceDate|InvoiceNo|Quantity|StockCode|UnitPrice|
+-------+----------+------------+---------+--------+---------+---------+
|     PX|    127269|1/12/10 8:28|   127269|       7|   22633V|     1.85|
|     PX|    227268|1/12/10 8:34|   227268|      38|   84879M|     1.69|
|     PX|    227268|1/12/10 8:34|   227268|       7|   22748P|      2.1|
|     PX|    227268|1/12/10 8:34|   227268|       9|   22749K|     3.75|
|     PX|    227268|1/12/10 8:34|   227268|       2|   22622G|     9.95|
|     PX|    227268|1/12/10 8:34|   227268|       4|   48187N|     7.95|
|     PX|    327267|1/12/10 8:34|   327267|       3|   22914W|     4.95|
|     PX|    162738|1/12/10 9:02|   162738|       7|  85123AY|     2.55|
|     PX|    162738|1/12/10 9:02|   162738|       7|   71053R|     3.39|
|     PX|    162738|1/12/10 9:02|   162738|       9|  84406BH|     2.75|
|     PX|    162738|1/12/10 9:02|   162738|       7

## 3. Exploration

### 3.1  Total row

In [5]:
from pyspark.sql.functions import *
print('Total row of train data :',df_train.select(count('CustomerID')).head()[0])
#df_train.select(count('CustomerID')).show()
print('Total row of test data :', df_test.select(count('CustomerID')).head()[0])
#df_test.select(count('CustomerID')).show()

Total row of train data : 330575
Total row of test data : 103097


### 3.2 Total Customer

In [6]:
# Find number of customer in train & test data
df1 = spark.sql('SELECT CustomerID FROM df_train GROUP BY CustomerID')
df2 = spark.sql('SELECT CustomerID FROM df_test GROUP BY CustomerID')
print('Number of customer in train data :' , df1.select(count('CustomerID')).head()[0]) 
#df1.select(count('CustomerID')).show()
print('Number of customer in test data :' , df2.select(count('CustomerID')).head()[0])
#df2.select(count('CustomerID')).show()

Number of customer in train data : 972
Number of customer in test data : 628


### 3.4 StockCode

In [7]:
# Find number of StockCode
df = spark.sql('SELECT StockCode, COUNT(StockCode) FROM df_train GROUP BY StockCode ORDER BY COUNT(StockCode) DESC')
print('Number of StockCode :' , df.select(count('StockCode')).head()[0]) 
df.show(50)

Number of StockCode : 3810
+---------+----------------+
|StockCode|count(StockCode)|
+---------+----------------+
|  85123AY|            1399|
|   22423U|            1344|
|  85099BJ|            1320|
|   47566Y|            1049|
|   20725N|            1001|
|   22720A|             938|
|   22197J|             906|
|   84879M|             876|
|   21212D|             866|
|   22383K|             831|
|   20727V|             820|
|   22457C|             805|
|   22469S|             788|
|   23203B|             775|
|   22386V|             769|
|   22086O|             763|
|   22960K|             757|
|   20728Z|             750|
|   22961C|             747|
|   23298Q|             745|
|   21931F|             741|
|   22382I|             740|
|   22411U|             736|
|   22666I|             728|
|   23209X|             715|
|   22384W|             689|
|   22699D|             688|
|   23206T|             679|
|   22727B|             674|
|   22993E|             673|
|   20726C|     

## 4. Create New Data  to Define Group of Customer

#### Select frist 30 popular item

In [8]:
# List of Customer
df_ID = spark.sql('SELECT CustomerID FROM df_train GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "85123AY" by CustomerID
df1 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f1 FROM df_train WHERE StockCode = "85123AY" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22423U" by CustomerID
df2 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f2 FROM df_train WHERE StockCode = "22423U" GROUP BY CustomerID ORDER BY CustomerID ASC')
 
# Purchase Product "85099BJ" by CustomerID
df3 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f3 FROM df_train WHERE StockCode = "85099BJ" GROUP BY CustomerID ORDER BY CustomerID ASC')
 
# Purchase Product "47566Y" by CustomerID
df4 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f4 FROM df_train WHERE StockCode = "47566Y" GROUP BY CustomerID ORDER BY CustomerID ASC')
 
# Purchase Product "20725N" by CustomerID
df5 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f5 FROM df_train WHERE StockCode = "20725N" GROUP BY CustomerID ORDER BY CustomerID ASC')
 
# Purchase Product "22720A" by CustomerID
df6 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f6 FROM df_train WHERE StockCode = "22720A" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22197J" by CustomerID
df7 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f7 FROM df_train WHERE StockCode = "22197J" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "84879M" by CustomerID
df8 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f8 FROM df_train WHERE StockCode = "84879M" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "21212D" by CustomerID
df9 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f9 FROM df_train WHERE StockCode = "21212D" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22383K" by CustomerID
df10 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f10 FROM df_train WHERE StockCode = "22383K" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "20727V" by CustomerID
df11 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f11 FROM df_train WHERE StockCode = "20727V" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22457C" by CustomerID
df12 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f12 FROM df_train WHERE StockCode = "22457C" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22469S" by CustomerID
df13 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f13 FROM df_train WHERE StockCode = "22469S" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "23203B" by CustomerID
df14 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f14 FROM df_train WHERE StockCode = "23203B" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22386V" by CustomerID
df15 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f15 FROM df_train WHERE StockCode = "22386V" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22086O" by CustomerID
df16 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f16 FROM df_train WHERE StockCode = "22086O" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22960K" by CustomerID
df17 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f17 FROM df_train WHERE StockCode = "22960K" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "20728Z" by CustomerID
df18 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f18 FROM df_train WHERE StockCode = "20728Z" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22961C" by CustomerID
df19 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f19 FROM df_train WHERE StockCode = "22961C" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "23298Q" by CustomerID
df20 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f20 FROM df_train WHERE StockCode = "23298Q" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "21931F" by CustomerID
df21 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f21 FROM df_train WHERE StockCode = "21931F" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22382I" by CustomerID
df22 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f22 FROM df_train WHERE StockCode = "22382I" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22411U" by CustomerID
df23 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f23 FROM df_train WHERE StockCode = "22411U" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22666I" by CustomerID
df24 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f24 FROM df_train WHERE StockCode = "22666I" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "23209X" by CustomerID
df25 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f25 FROM df_train WHERE StockCode = "23209X" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22384W" by CustomerID
df26 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f26 FROM df_train WHERE StockCode = "22384W" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22699D" by CustomerID
df27 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f27 FROM df_train WHERE StockCode = "22699D" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "23206T" by CustomerID
df28 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f28 FROM df_train WHERE StockCode = "23206T" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22727B" by CustomerID
df29 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f29 FROM df_train WHERE StockCode = "22727B" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22993E" by CustomerID
df30 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f30 FROM df_train WHERE StockCode = "22993E" GROUP BY CustomerID ORDER BY CustomerID ASC')


    

In [11]:
new_df = df_ID.join(df1, 'CustomerID', 'left_outer')
new_df = new_df.join(df2, 'CustomerID', 'left_outer')
new_df = new_df.join(df3, 'CustomerID', 'left_outer')
new_df = new_df.join(df4, 'CustomerID', 'left_outer')
new_df = new_df.join(df5, 'CustomerID', 'left_outer')
new_df = new_df.join(df6, 'CustomerID', 'left_outer')
new_df = new_df.join(df7, 'CustomerID', 'left_outer')
new_df = new_df.join(df8, 'CustomerID', 'left_outer')
new_df = new_df.join(df9, 'CustomerID', 'left_outer')
new_df = new_df.join(df10, 'CustomerID', 'left_outer')
new_df = new_df.join(df11, 'CustomerID', 'left_outer')
new_df = new_df.join(df12, 'CustomerID', 'left_outer')
new_df = new_df.join(df13, 'CustomerID', 'left_outer')
new_df = new_df.join(df14, 'CustomerID', 'left_outer')
new_df = new_df.join(df15, 'CustomerID', 'left_outer')
new_df = new_df.join(df16, 'CustomerID', 'left_outer')
new_df = new_df.join(df17, 'CustomerID', 'left_outer')
new_df = new_df.join(df18, 'CustomerID', 'left_outer')
new_df = new_df.join(df19, 'CustomerID', 'left_outer')
new_df = new_df.join(df20, 'CustomerID', 'left_outer')
new_df = new_df.join(df21, 'CustomerID', 'left_outer')
new_df = new_df.join(df22, 'CustomerID', 'left_outer')
new_df = new_df.join(df23, 'CustomerID', 'left_outer')
new_df = new_df.join(df24, 'CustomerID', 'left_outer')
new_df = new_df.join(df25, 'CustomerID', 'left_outer')
new_df = new_df.join(df26, 'CustomerID', 'left_outer')
new_df = new_df.join(df27, 'CustomerID', 'left_outer')
new_df = new_df.join(df28, 'CustomerID', 'left_outer')
new_df = new_df.join(df29, 'CustomerID', 'left_outer')
new_df = new_df.join(df30, 'CustomerID', 'left_outer')

# Replace null with 0
new_df = new_df.na.fill(0)
print('Check Number of CustomerID and Varaible')
print('Number of customer ;', new_df.select(count('CustomerID')).head()[0])


Check Number of CustomerID and Varaible
Number of customer ; 972


## 5. Clustering

In [12]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer
from pyspark.ml.clustering import KMeans
from pyspark.ml.clustering import KMeansModel
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import PCA


In [13]:
assembler = VectorAssembler(inputCols=['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10',\
                                       'f11','f12','f13','f14','f15','f16','f17','f18','f19','f20',\
                                       'f21','f22','f23','f24','f25','f26','f27','f28','f29','f30'],\
                            outputCol="features")
normalizer = Normalizer(inputCol='features' , outputCol='features_norm', p=1.0)

### 5.1 Find K use PCA 

In [208]:
# These line use to find PCA
pca = PCA(k=3, inputCol= 'features', outputCol="pcaFeatures")
pipeline_PCA = Pipeline(stages=[assembler, normalizer, pca])
model_PCA = pipeline_PCA.fit(new_df)

result_PCA = model_PCA.transform(new_df).select("pcaFeatures")
result_PCA.show(truncate=False)


### 5.2 K-mean 

In [14]:
# setK = 
kmeans = KMeans(maxIter=20, predictionCol="cluster").setK(30).setSeed(1)  # Initialize model
pipeline_Kmean = Pipeline(stages=[assembler, normalizer, kmeans])
model_Kmean = pipeline_Kmean.fit(new_df)

result_Kmean = model_Kmean.transform(new_df)
result_Kmean.groupby('cluster').count().show(30)

+-------+-----+
|cluster|count|
+-------+-----+
|     28|   16|
|     27|  163|
|     26|    2|
|     12|    2|
|     22|    9|
|      1|  569|
|     13|    1|
|     16|    1|
|      6|    1|
|      3|    1|
|     20|   32|
|      5|    1|
|     19|    3|
|     15|   11|
|     17|   19|
|      9|    2|
|      4|    4|
|      8|   19|
|     23|   65|
|      7|    7|
|     10|    2|
|     25|   26|
|     24|    2|
|     29|    4|
|     21|    1|
|     11|    1|
|     14|    1|
|      2|    3|
|      0|    1|
|     18|    3|
+-------+-----+



## 6. Save New train
new train is train with cluster (save to data asset)


In [43]:
# 1. Need to transform data frame 
import pandas as pd
df_cluster = result_Kmean['CustomerID', 'cluster']
new_train = df_train.join(df_cluster, 'CustomerID', 'left_outer')
new_train = new_train.toPandas()


In [47]:
# The code was removed by Watson Studio for sharing.

{'asset_id': '3e96d903-9d81-4e4f-a208-b89879e457d0',
 'bucket_name': 'recommendationengine-donotdelete-pr-vknxjbiqabtc3s',
 'file_name': 'new_train.csv',
 'message': 'File saved to project storage.'}

## 7. Make New Test 

### 7.1 Load test data

In [50]:
df_test = spark.read\
    .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
    .option('header', 'true')\
    .load(cos.url('test.csv', 'recommendationengine-donotdelete-pr-vknxjbiqabtc3s'))

df_test.createOrReplaceTempView('df_test')

### 7.2 Transform data

In [51]:
# List of Customer
df_ID = spark.sql('SELECT CustomerID FROM df_test GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "85123AY" by CustomerID
df1 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f1 FROM df_test WHERE StockCode = "85123AY" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22423U" by CustomerID
df2 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f2 FROM df_test WHERE StockCode = "22423U" GROUP BY CustomerID ORDER BY CustomerID ASC')
 
# Purchase Product "85099BJ" by CustomerID
df3 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f3 FROM df_test WHERE StockCode = "85099BJ" GROUP BY CustomerID ORDER BY CustomerID ASC')
 
# Purchase Product "47566Y" by CustomerID
df4 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f4 FROM df_test WHERE StockCode = "47566Y" GROUP BY CustomerID ORDER BY CustomerID ASC')
 
# Purchase Product "20725N" by CustomerID
df5 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f5 FROM df_test WHERE StockCode = "20725N" GROUP BY CustomerID ORDER BY CustomerID ASC')
 
# Purchase Product "22720A" by CustomerID
df6 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f6 FROM df_test WHERE StockCode = "22720A" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22197J" by CustomerID
df7 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f7 FROM df_test WHERE StockCode = "22197J" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "84879M" by CustomerID
df8 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f8 FROM df_test WHERE StockCode = "84879M" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "21212D" by CustomerID
df9 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f9 FROM df_test WHERE StockCode = "21212D" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22383K" by CustomerID
df10 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f10 FROM df_test WHERE StockCode = "22383K" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "20727V" by CustomerID
df11 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f11 FROM df_test WHERE StockCode = "20727V" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22457C" by CustomerID
df12 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f12 FROM df_test WHERE StockCode = "22457C" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22469S" by CustomerID
df13 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f13 FROM df_test WHERE StockCode = "22469S" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "23203B" by CustomerID
df14 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f14 FROM df_test WHERE StockCode = "23203B" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22386V" by CustomerID
df15 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f15 FROM df_test WHERE StockCode = "22386V" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22086O" by CustomerID
df16 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f16 FROM df_test WHERE StockCode = "22086O" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22960K" by CustomerID
df17 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f17 FROM df_test WHERE StockCode = "22960K" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "20728Z" by CustomerID
df18 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f18 FROM df_test WHERE StockCode = "20728Z" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22961C" by CustomerID
df19 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f19 FROM df_test WHERE StockCode = "22961C" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "23298Q" by CustomerID
df20 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f20 FROM df_test WHERE StockCode = "23298Q" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "21931F" by CustomerID
df21 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f21 FROM df_test WHERE StockCode = "21931F" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22382I" by CustomerID
df22 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f22 FROM df_test WHERE StockCode = "22382I" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22411U" by CustomerID
df23 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f23 FROM df_test WHERE StockCode = "22411U" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22666I" by CustomerID
df24 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f24 FROM df_test WHERE StockCode = "22666I" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "23209X" by CustomerID
df25 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f25 FROM df_test WHERE StockCode = "23209X" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22384W" by CustomerID
df26 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f26 FROM df_test WHERE StockCode = "22384W" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22699D" by CustomerID
df27 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f27 FROM df_test WHERE StockCode = "22699D" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "23206T" by CustomerID
df28 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f28 FROM df_test WHERE StockCode = "23206T" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22727B" by CustomerID
df29 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f29 FROM df_test WHERE StockCode = "22727B" GROUP BY CustomerID ORDER BY CustomerID ASC')

# Purchase Product "22993E" by CustomerID
df30 = spark.sql('SELECT CustomerID, SUM(Quantity) AS f30 FROM df_test WHERE StockCode = "22993E" GROUP BY CustomerID ORDER BY CustomerID ASC')



In [52]:
new_df = df_ID.join(df1, 'CustomerID', 'left_outer')
new_df = new_df.join(df2, 'CustomerID', 'left_outer')
new_df = new_df.join(df3, 'CustomerID', 'left_outer')
new_df = new_df.join(df4, 'CustomerID', 'left_outer')
new_df = new_df.join(df5, 'CustomerID', 'left_outer')
new_df = new_df.join(df6, 'CustomerID', 'left_outer')
new_df = new_df.join(df7, 'CustomerID', 'left_outer')
new_df = new_df.join(df8, 'CustomerID', 'left_outer')
new_df = new_df.join(df9, 'CustomerID', 'left_outer')
new_df = new_df.join(df10, 'CustomerID', 'left_outer')
new_df = new_df.join(df11, 'CustomerID', 'left_outer')
new_df = new_df.join(df12, 'CustomerID', 'left_outer')
new_df = new_df.join(df13, 'CustomerID', 'left_outer')
new_df = new_df.join(df14, 'CustomerID', 'left_outer')
new_df = new_df.join(df15, 'CustomerID', 'left_outer')
new_df = new_df.join(df16, 'CustomerID', 'left_outer')
new_df = new_df.join(df17, 'CustomerID', 'left_outer')
new_df = new_df.join(df18, 'CustomerID', 'left_outer')
new_df = new_df.join(df19, 'CustomerID', 'left_outer')
new_df = new_df.join(df20, 'CustomerID', 'left_outer')
new_df = new_df.join(df21, 'CustomerID', 'left_outer')
new_df = new_df.join(df22, 'CustomerID', 'left_outer')
new_df = new_df.join(df23, 'CustomerID', 'left_outer')
new_df = new_df.join(df24, 'CustomerID', 'left_outer')
new_df = new_df.join(df25, 'CustomerID', 'left_outer')
new_df = new_df.join(df26, 'CustomerID', 'left_outer')
new_df = new_df.join(df27, 'CustomerID', 'left_outer')
new_df = new_df.join(df28, 'CustomerID', 'left_outer')
new_df = new_df.join(df29, 'CustomerID', 'left_outer')
new_df = new_df.join(df30, 'CustomerID', 'left_outer')

# Replace null with 0
new_df = new_df.na.fill(0)
print('Check Number of CustomerID and Varaible')
print('Number of customer ;', new_df.select(count('CustomerID')).head()[0])



Check Number of CustomerID and Varaible
Number of customer ; 628


### 7.3 Clustering

In [53]:
result_Kmean = model_Kmean.transform(new_df)
result_Kmean.groupby('cluster').count().show(30)

+-------+-----+
|cluster|count|
+-------+-----+
|     28|    7|
|     27|   48|
|     12|    1|
|     22|    7|
|      1|  492|
|      6|    1|
|     20|   16|
|     17|    6|
|      8|    2|
|     23|   30|
|      7|    4|
|     10|    1|
|     25|    6|
|     14|    1|
|      2|    1|
|      0|    3|
|     18|    2|
+-------+-----+



### 7.4 Save New Test

In [55]:
# 1. Need to transform data frame 
import pandas as pd
df_cluster = result_Kmean['CustomerID', 'cluster']
new_test = df_test.join(df_cluster, 'CustomerID', 'left_outer')
new_test = new_test.toPandas()


In [56]:
# The code was removed by Watson Studio for sharing.

{'asset_id': '72e84155-caf5-4754-9504-ae4632cce35b',
 'bucket_name': 'recommendationengine-donotdelete-pr-vknxjbiqabtc3s',
 'file_name': 'new_test.csv',
 'message': 'File saved to project storage.'}

## 8. Make Recommend Product for each Cluster 

In [15]:
result_Kmean

DataFrame[CustomerID: string, f1: double, f2: double, f3: double, f4: double, f5: double, f6: double, f7: double, f8: double, f9: double, f10: double, f11: double, f12: double, f13: double, f14: double, f15: double, f16: double, f17: double, f18: double, f19: double, f20: double, f21: double, f22: double, f23: double, f24: double, f25: double, f26: double, f27: double, f28: double, f29: double, f30: double, features: vector, features_norm: vector, cluster: int]

In [23]:
df_cluster = result_Kmean['CustomerID', 'cluster']

In [24]:
df_train_1 = df_train.join(df_cluster, 'CustomerID', 'left_outer')

In [26]:
df_train_1.show(50)

+----------+---------+--------+--------------+---------+-------+---------+-------+
|CustomerID|InvoiceNo|Quantity|   InvoiceDate|UnitPrice|Country|StockCode|cluster|
+----------+---------+--------+--------------+---------+-------+---------+-------+
|    147348|   147348|       9| 3/12/10 14:30|     1.65|     GJ|   22331U|     27|
|    147348|   147348|      14| 3/12/10 14:30|      2.1|     GJ|   22865Y|     27|
|    147348|   147348|       4| 3/12/10 14:30|      8.5|     GJ|   22171T|     27|
|    147348|   147348|       4| 3/12/10 14:30|     5.95|     GJ|   22926R|     27|
|    147348|   147348|       3| 3/12/10 14:30|     4.95|     GJ|   22914W|     27|
|    147348|   147348|      14| 3/12/10 14:30|     1.69|     GJ|  84849DL|     27|
|    147348|   147348|       7| 3/12/10 14:30|     2.95|     GJ|   22445C|     27|
|    147348|   147348|      12| 3/12/10 14:30|     0.85|     GJ|   16016H|     27|
|    147348|   147348|      -1|  6/1/11 18:08|    10.95|     PX|   21843C|     27|
|   

In [27]:
df_train_1.select(count('CustomerID')).show()

+-----------------+
|count(CustomerID)|
+-----------------+
|           330575|
+-----------------+



In [30]:
df_train_1.createOrReplaceTempView('df_train_1')

In [33]:
df_train_1

DataFrame[CustomerID: string, InvoiceNo: string, Quantity: string, InvoiceDate: string, UnitPrice: string, Country: string, StockCode: string, cluster: int]

In [41]:
cluster = '1'
recommend = spark.sql('SELECT StockCode, SUM(Quantity) AS cluster0 FROM df_train_1 WHERE cluster = '+ cluster + ' GROUP BY StockCode ORDER BY SUM(Quantity) DESC')
recommend.show()

+---------+--------+
|StockCode|cluster0|
+---------+--------+
|   84077K| 11532.0|
|   16014F|  7964.0|
|   22492E|  6546.0|
|   15036Z|  6481.0|
|   22616B|  5326.0|
|   17003M|  5148.0|
|   22178V|  4771.0|
|   84879M|  4710.0|
|   21703M|  4568.0|
|   23084M|  4383.0|
|   84946F|  4362.0|
|   21790Y|  4128.0|
|   21212D|  4082.0|
|   22086O|  3974.0|
|   84945I|  3947.0|
|   84347V|  3901.0|
|   22065D|  3631.0|
|  85123AY|  3600.0|
|   21915K|  3475.0|
|   21977R|  3468.0|
+---------+--------+
only showing top 20 rows



In [41]:
StockCode_list = df.select('StockCode').collect()

In [132]:
StockCode_list[0].StockCode

'85123AY'

In [135]:
a= [row.StockCode for row in StockCode_list]

In [136]:
a

['85123AY',
 '22423U',
 '85099BJ',
 '47566Y',
 '20725N',
 '22720A',
 '22197J',
 '84879M',
 '21212D',
 '22383K',
 '20727V',
 '22457C',
 '22469S',
 '23203B',
 '22386V',
 '22086O',
 '22960K',
 '20728Z',
 '22961C',
 '23298Q',
 '21931F',
 '22382I',
 '22411U',
 '22666I',
 '23209X',
 '22384W',
 '22699D',
 '23206T',
 '22727B',
 '22993E',
 '20726C',
 '22178V',
 '22697A',
 '82482D',
 '20724F',
 '23084M',
 '22077P',
 '22726C',
 '23199M',
 '22139Q',
 '21080R',
 '22470E',
 '85099CQ',
 '21034O',
 '22138G',
 '84946F',
 '23201S',
 '21790Y',
 '23301G',
 '22629C',
 '23245W',
 '21929W',
 '20712E',
 '22355U',
 '21232B',
 '23202Z',
 '20914Q',
 '20719E',
 '22112V',
 '21977R',
 '22722E',
 '22630B',
 '82494LP',
 '85099FC',
 '22910D',
 '21731V',
 '21181K',
 '84991R',
 '21928E',
 '23355Z',
 '22090X',
 '22698C',
 '22111I',
 '21754X',
 '21175V',
 '22558H',
 '21733F',
 '23207R',
 '22728B',
 '84978X',
 '22356T',
 '23300F',
 '22379G',
 '20723U',
 '23344R',
 '22326L',
 '22624O',
 '21930O',
 '22659F',
 '22467G',
 '226

In [137]:
test_df = spark.createDataFrame(a)

TypeError: Can not infer schema for type: <class 'str'>

## 4. Make New Data frame